In [ ]:
import pymysql
from faker import Faker
import random

# Faker 객체 초기화
fake = Faker()

# 데이터베이스 연결 설정
conn = pymysql.connect(
    host='localhost',  # 데이터베이스 서버 주소
    user='root',       # 데이터베이스 사용자 이름
    password='931124', # 데이터베이스 비밀번호
    db='airbnb_db',    # 데이터베이스 이름
    charset='utf8mb4',
    cursorclass=pymysql.cursors.DictCursor
)

# Products 테이블을 위한 더미 데이터 생성
def generate_product_data(n):
    return [
        (
            f"{fake.word().capitalize()} {fake.word().capitalize()}",
            round(random.uniform(10, 100), 2),
            random.randint(10, 100),
            fake.date_time_this_year()
        )
        for _ in range(n)
    ]

# Customers 테이블을 위한 더미 데이터 생성
def generate_customer_data(n):
    return [
        (
            fake.name(),
            fake.unique.email(),
            fake.address(),
            fake.date_time_this_year()
        )
        for _ in range(n)
    ]

# Orders 테이블을 위한 더미 데이터 생성
def generate_order_data(n, customer_ids):
    return [
        (
            random.choice(customer_ids),
            fake.date_time_this_year(),
            round(random.uniform(20, 500), 2)
        )
        for _ in range(n)
    ]

# 데이터 삽입 함수
def insert_data():
    with conn.cursor() as cursor:
        # Products 데이터 삽입
        products_data = generate_product_data(10)
        products_sql = "INSERT INTO Products (productName, price, stockQuantity, createDate) VALUES (%s, %s, %s, %s)"
        cursor.executemany(products_sql, products_data)
        conn.commit()

        # Customers 데이터 삽입
        customers_data = generate_customer_data(5)
        customers_sql = "INSERT INTO Customers (customerName, email, address, createDate) VALUES (%s, %s, %s, %s)"
        cursor.executemany(customers_sql, customers_data)
        conn.commit()

        # Orders 데이터 삽입
        cursor.execute("SELECT customerID FROM Customers")
        customer_ids = [row['customerID'] for row in cursor.fetchall()]
        orders_data = generate_order_data(15, customer_ids)
        orders_sql = "INSERT INTO Orders (customerID, orderDate, totalAmount) VALUES (%s, %s, %s)"
        cursor.executemany(orders_sql, orders_data)
        conn.commit()

# 실행
try:
    insert_data()
    print("데이터 삽입 완료")
finally:
    conn.close()
